In [1]:
import abjad

NOTE: The Pärt demo requires abjad-ext-tonality


In [ ]:
'''
Today we worked on the ferneyhough demo.
It turns out that abjad's documents are a bit out of date.

They have new tuplet functionality, independent of the fernyhough demo.
Also, if you look at the Bartok demo and see abjad.Measure, that doesn't exist anymore.
Abjad has no measures.

It basically has abstract "container" types like Staff.
And concrete "leaf" types like notes.

Skim these

http://abjad.mbrsi.org/reference_manual/containers.html
http://abjad.mbrsi.org/api/abjad/core/Tuplet.html
'''

In [ ]:
'''
First, just make a tuplet with a given multiplier and pitch/duration sequence explicitly

Tuplet(multiplier, string)
The multiplier (2,3) means 3:2
The string consists of numbers for duration and commas for octave.
'''

t1 = abjad.Tuplet((2, 3), "c'8 d'8 e'8")
abjad.show(t1)
'''
or display just rhythm
'''
staff = abjad.Staff([t1],lilypond_type='RhythmicStaff',)
abjad.show(staff) 

In [ ]:
'''
Next, we can do tuplets from a duration.
Note that (1,8) is a duration, not a ratio.
if you don't put a number after d,e,f,g, they
will default to the same number 32 as set after c.
'''
t1 = abjad.Tuplet.from_duration((1,8), "c'32 d' e' f' g'")
staff = abjad.Staff([t1],lilypond_type='RhythmicStaff')
abjad.show(staff) 
'''
We can fix the beaming later.
'''

In [15]:
'''
Next, let's use the function we were working with earlier,
Tuplet from duration and ratio. Turns out there is a newer
version of it that isn't specific to the Ferneyhough demo
and offers more flexibility.
'''
t1 = abjad.Tuplet.from_duration_and_ratio(
        abjad.Duration(1, 4),
        abjad.Ratio((1,2))
    )

staff = abjad.Staff([t1],lilypond_type='RhythmicStaff')
abjad.show(staff) 

'''
Lets specify a different ratio
negatives for rests
'''
t2 = abjad.Tuplet.from_duration_and_ratio(
        abjad.Duration(1, 4),
        abjad.Ratio((-1,2,2))
    )


t3 = abjad.Tuplet.from_duration_and_ratio(
        abjad.Duration(1, 4),
        abjad.Ratio((2,-1,2))
    )


staff = abjad.Staff([t2,t3],lilypond_type='RhythmicStaff')

'''
Now be careful. Staff is a container.
It has 6 leaves, the members of the two tuplets
We can attach time signatures at leaves.

'''
leaves = abjad.select(staff).leaves()
time_signature = abjad.TimeSignature((1, 4))
abjad.attach(time_signature, leaves[0])
abjad.show(staff) 

6
r16
c'8


In [16]:
'''
Nested Tuplets

Let's make a tuple, delete the first part,
and replace with another tuple that fits the duration
of the deleted part.
'''

t1 = abjad.Tuplet.from_duration_and_ratio(
        abjad.Duration(1, 4),
        abjad.Ratio((1,2))
    )
t1.pop(0)
t2 = abjad.Tuplet.from_duration((1,8), "c'32 d' e' f' g'")
t1.insert(0, t2)
staff = abjad.Staff([t1],lilypond_type='RhythmicStaff',)
abjad.show(staff) 

In [17]:
'''
Here's the demo from earlier.
http://abjad.mbrsi.org/literature_examples/ferneyhough.html
'''

import abjad
from abjad.demos.ferneyhough import FerneyhoughDemo
ferneyhough = FerneyhoughDemo()
make_nested_tuplet = ferneyhough.make_nested_tuplet

def make_row_of_nested_tuplets(duration, ratio, column_count):
    tups = []
    assert column_count > 0
    for i in range(column_count):
        new_tup = make_nested_tuplet(quarter_dur, ratio, i+1)
        if new_tup.multiplier==1:
            new_tup.hide = True
        tups.append(new_tup)
    return tups

def make_rows_of_nested_tuplets(duration, row_count, column_count, increasing_side='right'):
    assert row_count > 0
    assert increasing_side in ['left','right'],"increasing side should be left or right"
    rows = []
    for r in range(row_count):
        if increasing_side == 'right':
            ratio = (1, r+1)
        else:
            ratio = (r+1, 1)
        new_row = make_row_of_nested_tuplets(duration, ratio, column_count)
        rows.append(new_row)
    return rows
    
score = abjad.Score()
quarter_dur = abjad.Duration(1, 4)
ROWS = 5
COLUMNS = 3
rows = make_rows_of_nested_tuplets(quarter_dur, ROWS, COLUMNS,increasing_side='right')
for row in rows:
    staff = abjad.Staff(row, lilypond_type='RhythmicStaff')
    score.append(staff)
abjad.show(score)